# 명함인식 OpenCV

In [1]:
# 모듈 있는지 확인
import sys
import cv2
import pytesseract 
from PIL import Image

pytesseract.pytesseract.tesseract_cmd= r'/Users/woodeem/opt/anaconda3/envs/rnd/bin/tesseract'

print(sys.version)
print(cv2.__version__)
pytesseract.image_to_string(Image.open('bizcardsample/SampleImg.jpeg'), lang='kor+eng', config='--psm 1 -c preserve_interword_spaces=1')


3.7.13 (default, Mar 28 2022, 07:24:34) 
[Clang 12.0.0 ]
4.5.5


'미0\n\nae\n공인중개사\n\n공인중개사 깁 a =\n\nTEL —010.2235.6675\n\nFAX = 051.524.6675\n\nE-MAIL junghun12@naver.com\n\nADD _ 부산 해운대구 재송로 32번길 17\n\n \n\x0c'

## Biz Logic
1. 이미지 읽기
2. 이미지로부터 명함 엣지 추출
3. 이미지 평면화
4. 스캔한 효과를 줘서 조명 영향 제거
5. OCR 엔진을 이용해 글자 인식

In [2]:
# OpenCV - 이미지 읽기, 쓰기 및 표시하기

import numpy as np
import cv2

def handle_image():
    imgfile = 'bizcardsample/HairSample.png'
    img = cv2.imread(imgfile, cv2.IMREAD_COLOR)
    
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    handle_image()

In [3]:
# OpenCV - 이미지 읽기, 쓰기 및 표시하기

import numpy as np
import cv2

def handle_image():
    imgfile = 'bizcardsample/HairSample.png'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    cv2.imshow('image', img)
    k = cv2.waitKey(0)
    
    # wait for ESC key to exit
    if k == 27:
        cv2.destroyAllWindows()
        cv2.waitKey(1)
    # wait for 's' key to save and exit
    elif k == ord('s'):
        cv2.imwrite('bizcardsample/grayImage.png', img)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
if __name__ == '__main__':
    handle_image()

### 명함 외곽 추출하기

In [4]:
# OpenCV 도형 외곽 추출하기
import numpy as np
import cv2
from matplotlib import pyplot as plt
def contour():
    imgfile = 'bizcardsample/HairSample.png'
    img = cv2.imread(imgfile)
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    edge = cv2.Canny(imgray, 100, 200)
    contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.imshow('edge', edge)
    cv2.drawContours(img, contours, -1, (0, 255, 0), 1)
    cv2.imshow('Contour', img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    contour()

### 명함이 구겨져 있거나 정확한 직사각형이 아니면, 도형을 근사해서 계산함으로써 과적합(Overfitting) 방지함

In [5]:
# OpenCV - 도형 외곽 추출하기
import numpy as np
import cv2
from matplotlib import pyplot as plt 

def contour_approx():
    imgfile = 'bizcardsample/HairSample.png'
    img = cv2.imread(imgfile)
    img2 = img.copy()
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    edge = cv2.Canny(imgray, 100, 200)
    contours, hierarchy = cv2.findContours(edge, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    print(len(contours))
    print(contours[0].shape)
    
    cnt = contours[0]
    cv2.drawContours(img, [cnt], 0, (0, 255, 0), 3)
    
    epsilon = 0.1 * cv2.arcLength(cnt, True)
    
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    
    cv2.drawContours(img2, [approx], 0, (0, 255, 0), 3)
    
    cv2.imshow('Contour', img)
    cv2.imshow('Approx', img2)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    contour_approx()

247
(25, 1, 2)


### 투영변환 : 외곽으로 그려진 영역을 반듯하게 변환하기

In [6]:
# OpenCV - 투영변환 구현하기
import numpy as np 
import cv2

def warpAffine():
    img = cv2.imread('bizcardsample/HairSample.png')
    
    pts1 = np.float32([[50,50], [200,50], [20, 200]])
    pts2 = np.float32([[70,100], [220,50], [150, 250]])
    
    M = cv2.getAffineTransform(pts1, pts2)
    
    result = cv2.warpAffine(img, M, (500, 300))
    
    cv2.imshow('original', img)
    cv2.imshow('Affine Transform', result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    warpAffine()

In [7]:
import numpy as np
import cv2

def warpPerspective():
    img = cv2.imread('bizcardsample/ObidjonNameCard.jpeg')
    
    topLeft = [127, 157]
    topRight = [448, 152]
    bottomRight = [579, 526]
    bottomLeft = [54, 549]
    
    pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
    
    w1 = abs(bottomRight[0] - bottomLeft[0])
    w2 = abs(topRight[0] - topLeft[0])
    h1 = abs(topRight[1] - bottomRight[1])
    h2 = abs(topLeft[1] - bottomLeft[1])
    
    minWidth = min([w1, w2])
    minHeight = min([h1, h2])
    
    pts2 = np.float32([[0,0], [minWidth-1,0], [minWidth-1, minHeight-1], [0,minHeight-1]])
    
    M = cv2.getPerspectiveTransform(pts1, pts2)
    
    result = cv2.warpPerspective(img, M, (int(minWidth), int(minHeight)))
    
    cv2.imshow('original', img)
    cv2.imshow('Warp Transform', result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__':
    warpPerspective()

### 스캔한 효과를 줘서 조명의 영향 제거하기

In [8]:
# OpenCV - 스캔 효과 주기
import numpy as np
import cv2

# Callback Function for Trackbar (but do not any work)
def nothing(x):
    pass

def global_threshold():
    imgfile = 'bizcardsample/HairSample.png'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    r = 600.0 / img.shape[0]
    dim = (int(img.shape[1] * r), 600)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    WindowName = "Window"
    TrackbarName = "Threshold"
    
    cv2.namedWindow(WindowName)
    cv2.createTrackbar(TrackbarName, WindowName, 50, 255, nothing)
    Threshold = np.zeros(img.shape, np.uint8)
    
    while True:
        TrackbarPos = cv2.getTrackbarPos(TrackbarName, WindowName)
        cv2.threshold(img, TrackbarPos, 255, cv2.THRESH_BINARY, Threshold)
        cv2.imshow(WindowName, Threshold)
        
        k = cv2.waitKey(0)
        if k == 27:
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
    return

if __name__ == '__main__':
    global_threshold()

### Threshold 자동으로 구하기 & 조명 영향 줄이기

In [1]:
# OpenCV - 스캔 효과 주기 (2)
import cv2
def adaptive_threshold():
    imgfile = 'bizcardsample/HairSample.png'
    img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
    
    r = 600.0 / img.shape[0]
    dim = (int(img.shape[1] * r), 600)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    result_without_blur = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)
    result_with_blur = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)
    cv2.imshow('Without Blur', result_without_blur)
    cv2.imshow('With Blur', result_with_blur)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

if __name__ == '__main__' :
    adaptive_threshold()

## 🦾 명함인식 구현 🦾 

In [10]:
# Step1. Edge Detection
# 라이브러리 로드
import cv2
import numpy as np
from PIL import Image
import pytesseract

# 이미지 로드 후 높이 비율 맞추기
image = cv2.imread('bizcardsample/Akrom_Kor.jpeg')
orig = image.copy()
r = 800.0 / image.shape[0]
dim = (int(image.shape[1] * r), 800)
image = cv2.resize(image, dim, interpolation= cv2.INTER_AREA)

# Gray Scale 로 변환후 엣지 찾기
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
edged = cv2.Canny(gray, 20, 75)

# 원본 이미지를 보여주고, Edge 디텍션
print ("STEP1 : Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

# ---------------------------------
# Step2. Find Contours of Paper
# Edge 윤곽 찾기

(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) # findContours를 통해 contours들을 반환받음
cnts = sorted(cnts, key = cv2.contourArea, reverse=True)

# contour 찾기 반복(4개의 포인트)
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    
    if len(approx) == 4:
        screenCnt = approx
        break

# outlne 프린팅
print("STEP2 : Find Contours of Paper")
cv2.drawContours(image, cnts,  -1, (0, 255, 0), 1)
cv2.imshow("Outline", image)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

# ---------------------------------
# Step3. Apply Perspective Transform
# 이미지 똑바로 세우기
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32") # 4개 꼭지점 좌표 입력할 4x2 행렬 생성

    s = pts.sum(axis = 1)       # (x, y) 좌표에서 x+y 계산 (axis 0=열, 1=행)
    rect[0] = pts[np.argmin(s)] # x+y의 최대값 (topLeft)
    rect[2] = pts[np.argmax(s)] # x+y의 최소값 (bottomRight)

    diff = np.diff(pts, axis = 1)  # (x, y) 좌표에서 y-x 계산 (axis 0=열, 1=행)
    rect[1] = pts[np.argmin(diff)] # y-x의 최소값 (topRight)
    rect[3] = pts[np.argmax(diff)] # y-x의 최대값 (bottomLeft)
    return rect

rect = order_points(screenCnt.reshape(4, 2) / r)
(topLeft, topRight, bottomRight, bottomLeft) = rect

w1 = abs(bottomRight[0] - bottomLeft[0])    # 하단 폭
w2 = abs(topRight[0] - topLeft[0])          # 상단 폭
h1 = abs(topRight[1] - bottomRight[1])      # 우측 높이
h2 = abs(topLeft[1] - bottomLeft[1])        # 좌측 높이

maxWidth = int(max([w1, w2]))
maxHeight = int(max([h1, h2]))

dst = np.float32([[0, 0], [maxWidth, 0], [maxWidth, maxHeight], [0, maxHeight]]) # 변환될 크기만큼 행렬 생성
M = cv2.getPerspectiveTransform(rect, dst) # getPerspectiveTransform()함수를 통해서 나머지 픽셀을 옮기는 매트릭스 M에 반환
warped = cv2.warpPerspective(orig, M, (maxWidth, maxHeight)) # M을 warpPerspective()에 넣음으로써 최종적으로 반듯한 사각형으로 변환된 이미지를 받음


#원본과 스캔이미지 보기
print("Step3 : Apply Perspective transform")
cv2.imshow("Warped", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


# ---------------------------------
# Step3. Apply Adaptive Threshold
# 이미지 스캔효과주기
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)

print("Step4 : Apply Adaptive Threshold")
cv2.imshow("Original", orig)
cv2.imshow("Scanned", warped)
img2 = cv2.imwrite('samples/scannedImage.png', warped)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


def ocr_tesseract(img):
    im = Image.open(img)
    text = pytesseract.image_to_string(im, lang='kor+eng')
    im.show()
    print(text)
    
ocr_tesseract('samples/scannedImage.png')

STEP1 : Edge Detection
STEP2 : Find Contours of Paper
Step3 : Apply Perspective transform
Step4 : Apply Adaptive Threshold
때에 |.
consulting partner

사이드카몰로브 아크롬

연구원
기엽부설연구소

0312100

아이트멕스

rT

Mobile 010 5900 5434
Tel       02 365 2187
Fax       02 365 2093
akrom@i2max.co.kr

(주)아이투맥스
서울시 마포구 마포대로 137,
KPX' &} 7증 (04143)

{

i
3



In [8]:
import cv2
import os
import numpy as np

mser = cv2.MSER_create()
regions,_ = mser.detectRegions(gray)
clone = img.copy()
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
remove1 = []
for i,c1 in enumerate(hulls):
    x, y, w, h = cv2.boundingRect(c1)
    r1_start = (x, y)
    r1_end = (x+w, y+h)
    for j,c2 in enumerate(hulls):
        if i == j:
            continue
        x, y, w, h = cv2.boundingRect(c2)
        r2_start = (x, y)
        r2_end = (x+w, y+h)
        if r1_start[0]> r2_start[0] and r1_start[1] > r2_start[1] and r1_end[0] < r2_end[0] and r1_end[1] < r2_end[1]:
            remove1.append(i)

for j,cnt in enumerate(hulls):
    if j in remove1: continue
    x, y, w, h = cv2.boundingRect(cnt)
    margin = 10
    cv2.rectangle(clone, (x-margin, y-margin), (x + w + margin, y + h + margin), (0, 255, 0), 1)

cv2.imshow('mser', clone)
cv2.waitKey(0)

mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)

for j,cnt in enumerate(hulls):
    if j in remove1: continue
    x, y, w, h = cv2.boundingRect(cnt)
    margin = 10
    cv2.rectangle(mask, (x-margin, y-margin), (x + w + margin, y + h + margin), (255, 255, 255), -1)

text_only = cv2.bitwise_and(img, img, mask=mask)

cv2.imshow("text only", text_only)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

13

# 0406

In [168]:
# Step1. Edge Detection
# 라이브러리 로드
import cv2
import numpy as np
from PIL import Image
import pytesseract
import re

# 이미지 로드 후 높이 비율 맞추기
image_path = 'bizcardsample/Crystal.jpg'
image = cv2.imread(image_path)
orig = image.copy()
r = 800.0 / image.shape[0]
dim = (int(image.shape[1] * r), 800)
image = cv2.resize(image, dim, interpolation= cv2.INTER_AREA)


# Gray Scale 로 변환후 엣지 찾기
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
edged = cv2.Canny(gray, 20, 200)

# 원본 이미지를 보여주고, Edge 디텍션
print ("STEP1 : Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

# ---------------------------------
# Step2. Find Contours of Paper
# Edge 윤곽 찾기

(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) # findContours를 통해 contours들을 반환받음
cnts = sorted(cnts, key = cv2.contourArea, reverse=True)

# contour 찾기 반복(4개의 포인트)
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    
    if len(approx) == 4:
        screenCnt = approx
        break

# outlne 프린팅
print("STEP2 : Find Contours of Paper")
cv2.drawContours(image, cnts,  -1, (0, 255, 0), 1)
cv2.imshow("Outline", image)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


# ---------------------------------
# Step3. Apply Perspective Transform
# 이미지 똑바로 세우기
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32") # 4개 꼭지점 좌표 입력할 4x2 행렬 생성

    s = pts.sum(axis = 1)       # (x, y) 좌표에서 x+y 계산 (axis 0=열, 1=행)
    rect[0] = pts[np.argmin(s)] # x+y의 최대값 (topLeft)
    rect[2] = pts[np.argmax(s)] # x+y의 최소값 (bottomRight)

    diff = np.diff(pts, axis = 1)  # (x, y) 좌표에서 y-x 계산 (axis 0=열, 1=행)
    rect[1] = pts[np.argmin(diff)] # y-x의 최소값 (topRight)
    rect[3] = pts[np.argmax(diff)] # y-x의 최대값 (bottomLeft)
    return rect

rect = order_points(screenCnt.reshape(4, 2) / r)
(topLeft, topRight, bottomRight, bottomLeft) = rect

w1 = abs(bottomRight[0] - bottomLeft[0])    # 하단 폭
w2 = abs(topRight[0] - topLeft[0])          # 상단 폭
h1 = abs(topRight[1] - bottomRight[1])      # 우측 높이
h2 = abs(topLeft[1] - bottomLeft[1])        # 좌측 높이

maxWidth = int(max([w1, w2]))
maxHeight = int(max([h1, h2]))

dst = np.float32([[0, 0], [maxWidth, 0], [maxWidth, maxHeight], [0, maxHeight]]) # 변환될 크기만큼 행렬 생성
M = cv2.getPerspectiveTransform(rect, dst) # getPerspectiveTransform()함수를 통해서 나머지 픽셀을 옮기는 매트릭스 M에 반환
warped = cv2.warpPerspective(orig, M, (maxWidth, maxHeight)) # M을 warpPerspective()에 넣음으로써 최종적으로 반듯한 사각형으로 변환된 이미지를 받음

#///////////


#원본과 스캔이미지 보기
print("Step3 : Apply Perspective transform")
cv2.imshow("Warped", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


# ---------------------------------
# Step3. Apply Adaptive Threshold
# 이미지 스캔효과주기
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)

print("Step4 : Apply Adaptive Threshold")
cv2.imshow("Original", orig)
cv2.imshow("Scanned", warped)
img2 = cv2.imwrite('samples/scannedImage.png', warped)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

def ocr_tesseract(img):
    im = Image.open(img)
    text = pytesseract.image_to_string(im, lang='kor+eng')
    # im.show()
    if len(text) < 1 :
        
        print("하얀색이 아닌 배경에서 찍어주세요 😊")
    else:
        text = re.sub('[=,#/\?:^*\"※~ㆍ!』‘|\[\]`\'…》\”\“\’·]', ' ', text)
        text = list(filter(None, text.replace('\n',' ').split(' ')))
        
        # --------------------------------------
        print("-----명함주인 정보-----")
        biz_info ={"company": "", "email": "", "role": "", "region": "", "Mobile": "", "Tel":"", "Fax": ""}
        
        
        # Company
        comp_name = [s for s in text if "(주)" in s]
        if comp_name:
            biz_info['company'] = comp_name[0]
        else:
            biz_info['company'] = None
        
        #Email
        email_name = [s for s in text if "@" in s]
        if email_name:
            biz_info['email'] = email_name[0]
        else:
            biz_info['email'] = None
        
        #직급
        role_name = [s for s in text if "원" in s]
        if role_name:
            biz_info['role'] = role_name[0]
        else:
            biz_info['role'] = None
        
        #지역
        region_flag = ("시","군","구","로","빌딩", ",","층","호",")")
        regions = [" ".join([name for name in text if name.endswith(region_flag)])]
        if role_name:
            biz_info['region'] = regions[0]
        else:
            biz_info['region'] = None
        
        #Contacts
        if "Mobile" in text:
            mob_idx = text.index("Mobile")
            mob = ''.join(text[mob_idx+1:mob_idx+5])
            mob = [re.sub(r'[^0-9]', '', mob)][0]
            biz_info['Mobile'] = mob
        if "Tel" in text:    
            tel_idx = text.index("Tel")
            tel = ''.join(text[tel_idx+1:tel_idx+5])
            tel = [re.sub(r'[^0-9]', '', tel)][0]
            biz_info['Tel'] = tel
        if "Fax" in text:     
            fax_idx = text.index("Fax")
            fax = ''.join(text[fax_idx+1:fax_idx+5])
            fax = [re.sub(r'[^0-9]', '', fax)][0]
            biz_info['Fax'] = fax   
        
        print(biz_info)
        # print(text)
    
    
    
    
ocr_tesseract('samples/scannedImage.png')

STEP1 : Edge Detection
STEP2 : Find Contours of Paper
Step3 : Apply Perspective transform
Step4 : Apply Adaptive Threshold
-----명함주인 정보-----
{'company': '(주)아이투맥스', 'email': 'sjchoi@i2max.co.kr', 'role': '사원', 'region': '서울시 마포구 마포대로 ×빌딩 7층 (04143)', 'Mobile': '01071581385', 'Tel': '', 'Fax': '023652093'}


In [ ]:
import cv2
print(cv2.__version__)

4.5.5


In [ ]:
from PIL import Image
import pytesseract

def ocr_tesseract(img):
    im = Image.open(img)
    text = pytesseract.image_to_string(im, lang='kor+eng')
    im.show()
    print(text)

if __name__ == '__main__':
    ocr_tesseract()


TypeError: ocr_tesseract() missing 1 required positional argument: 'img'